### TensorFlow has an extensive amount of documentation, example notebooks and projects.   

 For a quick glance, check out Tim Spann's Introduction to TensorFlow RefCard
 
 - https://dzone.com/refcardz/introduction-to-tensorflow?chapter=1
 - https://www.tensorflow.org/
 - https://community.hortonworks.com/articles/118132/minifi-capturing-converting-tensorflow-inception-t.html
 - https://community.hortonworks.com/articles/174520/minifi-c-iot-cat-sensor.html
 - https://community.hortonworks.com/articles/103863/using-an-asus-tinkerboard-with-tensorflow-and-pyth.html
 - https://medium.com/ibm-data-science-experience/tagged/data-science


### TensorFlow in Production

For those interested in using TensorFlow in production with HDP and HDF, here are some resources.

- https://community.hortonworks.com/articles/178498/integrating-tensorflow-16-image-labelling-with-hdf.html
- https://community.hortonworks.com/articles/193868/integrating-keras-tensorflow-yolov3-into-apache-ni.html
- https://community.hortonworks.com/articles/80339/iot-capturing-photos-and-analyzing-the-image-with.html
- https://community.hortonworks.com/articles/183806/using-a-tensorflow-person-blocker-with-apache-nifi.html

![Photo for Inception](https://raw.githubusercontent.com/tspannhw/DWS-DeepLearning-CrashCourse/master/photo1.jpg "Photo")


In [4]:
# ==============================================================================
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================





#### Simple image classification with Inception
 
Run image classification with Inception trained on ImageNet 2012 Challenge dataset.
 
- This program creates a graph from a saved GraphDef protocol buffer
- then runs inference on an input JPEG image. 
- It outputs human readable strings of the top 5 predictions along with their probabilities.

https://tensorflow.org/tutorials/image_recognition/

In [5]:
# Imports of necessary libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import errno as _errno
import sys as _sys

# import tensorflow libraries for parsing flags
from tensorflow.python.platform import flags
from tensorflow.python.util.all_util import remove_undocumented
from tensorflow.python.util.tf_export import tf_export

# import argument parsing, operating system, date and math libraries
import argparse
import os.path
import re
import sys
import tarfile
import os
import datetime
import math


# numpy is for numeric processing
import numpy as np

# to load files over network
from six.moves import urllib

# the tensorflow python interface
import tensorflow as tf



In [7]:
# set logging levels
tf.logging.set_verbosity(tf.logging.ERROR)

# No TensorFlow Run Flags Required
FLAGS = None

# We need to download the inception pre-trained model if not done already.   Happens at most once.

# pylint: disable=line-too-long
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
# pylint: enable=line-too-long

In [9]:
# ----- Node Lookup Object
# ----- 
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  # initialize method
  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  # load the text descriptions
  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.
    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.
    Returns:
      dict from integer node ID to human-readable string.
    """
    if not tf.gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not tf.gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)


    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = tf.gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string


    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = tf.gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  # convert the node id to a string
  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]


# create the graph from the pre-trained stored binary model
def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile(os.path.join(
      FLAGS.model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')


# we are running the pretrained model
def run_inference_on_image(image):
  """Runs inference on an image.
  Args:
    image: Image file name.
  Returns:
    Nothing
  """

  # We load an image from the crash course github to classify
  # You could substitute for any JPG or PNG URL
  image_url = "https://raw.githubusercontent.com/tspannhw/DWS-DeepLearning-CrashCourse/master/photo1.jpg"
  req = urllib.request.Request(image_url)
  response = urllib.request.urlopen(req)
  image_data = response.read()

  # Creates graph from saved GraphDef.
  create_graph()

  # tensorflow session processing
  with tf.Session() as sess:
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    predictions = sess.run(softmax_tensor,
                           {'DecodeJpeg/contents:0': image_data})
    predictions = np.squeeze(predictions)


    # Creates node ID --> English string lookup.
    node_lookup = NodeLookup()
    
    # print jupyter
    print('Node ID\tDescription\tScore\n')

    # sort the predictions in order of score
    top_k = predictions.argsort()[-FLAGS.num_top_predictions:][::-1]

    # output all of our human string and scores
    for node_id in top_k:
      # get the printable string for that id
      human_string = node_lookup.id_to_string(node_id)
      # get the prediction score for that id
      score = predictions[node_id]
      # let's print it as a nice table in zeppelin
      print('{0}\t{1}\t{2}%\n'.format(str(node_id), str(human_string),  float(score) * 100.0))

# download and extract model if not done already
def maybe_download_and_extract():
  """Download and extract model tar file."""
  dest_directory = FLAGS.model_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
        pass
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    statinfo = os.stat(filepath)
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)

# let's run it all
def main(_):
  # download pretrained model if necessary
  maybe_download_and_extract()
  # name of our file on the local filesystem
  img_name = 'https://raw.githubusercontent.com/tspannhw/DWS-DeepLearning-CrashCourse/master/photo1.jpg'
  # run on that image
  run_inference_on_image(img_name)


In [10]:
# Run the TensorFlow NodeLookup object

# create arguments
parser = argparse.ArgumentParser()

# classify_image_graph_def.pb:
#   Binary representation of the GraphDef protocol buffer.
# imagenet_synset_to_human_label_map.txt:
#   Map from synset ID to a human readable string.
# imagenet_2012_challenge_label_map_proto.pbtxt:
#   Text representation of a protocol buffer mapping a label to synset ID.
parser.add_argument(
      '--model_dir',
      type=str,
      default='/tmp',
      help="""\
      Path to classify_image_graph_def.pb,
      imagenet_synset_to_human_label_map.txt, and
      imagenet_2012_challenge_label_map_proto.pbtxt.\
      """)
parser.add_argument(
      '--image_file',
      type=str,
      default='',
      help='Absolute path to image file.' )
parser.add_argument(
      '--num_top_predictions',
      type=int,
      default=5,
      help='Display this many predictions.')
FLAGS, unparsed = parser.parse_known_args()
#tf.app.run(main=main)
main(FLAGS)

Node ID	Description	Score

273	racer, race car, racing car	37.4601334333%

274	sports car, sport car	25.3520905972%

267	cab, hack, taxi, taxicab	11.1182719469%

268	convertible	9.85429733992%

271	minivan	3.2295178622%



You know have a sorted list of results of the classification.   The description is from a list of texts that can be updated.

The score is a %, with 100% being a perfect match.
